In [ ]:
!pip install control
!pip install drake

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
import control
from pydrake.systems.controllers import LinearQuadraticRegulator
from scipy.linalg import solve_continuous_are

# ===============================
# 1. DEFINE STATE SPACE SYSTEM
# ===============================
def continuous_time_linearized_system_dynamics(g, m, L, l, I_xx, I_yy, I_zz):
    """
        Initialize the state-space model of the quadrotor-pendulum system.

        Parameters:
        - g: Gravitational acceleration (m/s^2)
        - m: Mass of the quadrotor (kg)
        - L: Length of pendulum to center of mass (m)
        - l: Quadrotor center to rotor center (m)
        - I_xx, I_yy, I_zz: Inertia tensors of the quadrotor (kg·m²)

        Returns:
        - sysc: Continuous-time state-space system
    """

    # Number of states: 16
    # States:
    # 0: x, 1: y, 2: z
    # 3: alpha, 4: beta, 5: gamma
    # 6: x_dot, 7: y_dot, 8: z_dot
    # 9: alpha_dot, 10: beta_dot, 11: gamma_dot
    # 12: r, 13: s
    # 14: r_dot, 15: s_dot

    n = 16 # Number of states
    m_in = 4 # Number of inputs

    A = np.zeros((n, n))
    B = np.zeros((n, m_in))
    C = np.eye(n)  # output = input

    # Populate A matrix
    # Position derivatives
    A[0, 6] = 1    # x_dot
    A[1, 7] = 1    # y_dot
    A[2, 8] = 1    # z_dot

    # Angle derivatives
    A[3, 9] = 1    # alpha_dot
    A[4, 10] = 1   # beta_dot
    A[5, 11] = 1   # gamma_dot

    # Translational accelerations influenced by angles
    A[6, 3] = g/m  # x_ddot
    A[7, 4] = g/m  # y_ddot

    # Pendulum dynamics
    A[12, 14] = 1    # r_dot
    A[13, 15] = 1    # s_dot
    A[14, 12] = -g/L  # r_ddot
    A[14, 3] = g/L  # Coupling term
    A[15, 13] = -g/L      # s_ddtot
    A[15, 4] = g/L  # Coupling term

    # Populate B matrix
    # Control inputs: delta_a, omega_x, omega_y, omega_z
    B[8, 0] = 1 / m    # delta_a affects z_ddot
    B[9, 1] = 1 / I_xx  # omega_x affects alpha_ddot
    B[10, 2] = 1 / I_yy  # omega_y affects beta_ddot
    B[11, 3] = 1 / I_zz  # omega_z affects gamma_ddot

    # Create continuous-time state-space system
    sysc = control.StateSpace(A, B, C, np.zeros((n,m_in)))

    return sysc, A, B

def check_controllability(sysc):
    """
        Check if the given state-space system is controllable.

        Parameters:
        - sysc: Continuous-time state-space system

        Prints:
        - Controllability matrix rank and status
    """

    # Extract A and B matrices from the state-space object
    A = sysc.A
    B = sysc.B

    # Compute the controllability matrix
    ctrb_matrix = control.ctrb(A, B)

    # Calculate the rank of the controllability matrix
    rank_of_ctrb = np.linalg.matrix_rank(ctrb_matrix)

    # Number of states
    n_states = A.shape[0]

    print(f"Controllability Matrix Rank: {rank_of_ctrb} / {n_states}")

    if rank_of_ctrb < n_states:
        print("The system is not controllable and LQR cannot be applied.")
    else:
        print("The system is controllable and LQR can be applied.")

# ===============================
# 2. DISCRETIZE SYSTEM
# ===============================

def discretized_sys(sysc, h):
    # Discretize the system using Zero-Order Hold (ZOH)
    sysd = control.sample_system(sysc, h, method='zoh')

    A = sysd.A
    B = sysd.B
    C = sysd.C
    D = sysd.D

    return A,B,C,D

# ===============================
# 3. LINEAR QUADRATIC REGULATOR (LQR)
# ===============================

def lqr_normal(A,B,Q,R,x):
    S = solve_continuous_are(A, B, Q, R)
    K = -np.linalg.inv(R) @ B.T @ S
    u = - K @ x;

    return u

def lqr_drake(K,x,T_steps):
  # Find optimal control input for the current state
  for k in range(T_steps):
    # Current state
    current_state = x[:, k].reshape(-1, 1)

    # Compute control input
    u[:, k] = (-K @ current_state).flatten()

    # Apply control action and compute next state
    x[:, k+1] = A @ x[:, k] + B @ u[:, k]

  return u


if __name__ == '__main__':
  # Physical constants and system parameters
  g = 9.81       # Gravitational acceleration (m/s^2)
  m = 0.5        # Mass of the quadrotor (kg)
  L = 0.565      # Length of pendulum to center of mass (m)
  l = 0.17       # Quadrotor center to rotor center (m)
  I_yy = 3.2e-3  # Inertia around y-axis (kg·m²)
  I_xx = I_yy    # Assuming I_xx = I_yy
  I_zz = 5.5e-3  # Inertia around z-axis (kg·m²)

  # Simulation parameters
  h = 0.1        # Sampling time
  simTime = 10   # Simulation time
  T_steps = int(simTime / h)
  t = np.linspace(0, simTime, T_steps+1)  # Time vector

  # Initialize the continuous-time state-space system
  sysc,A,B = continuous_time_linearized_system_dynamics(g, m, L, l, I_xx, I_yy, I_zz)

  # Check controllability
  check_controllability(sysc)

  # # Discretize system
  # A,B,C,D = discretized_sys(sysc,h)

  # Initialize state vector
  x0 = np.array([0.02, 0, 0.01, 0, 0, 0, 0.02, 0, 0.04, 0, 0, 0, 0, 0, 0, 0]).reshape(-1, 1)  # Column vector

  # Define Q and R matrices for LQR
  Q = np.eye(A.shape[0])  # State cost
  R = np.eye(B.shape[1])  # Control input cost

  # Initialize state, control
  x = np.zeros((A.shape[0], T_steps+1))  # States
  u = np.zeros((B.shape[1], T_steps))    # Control inputs

  # Set initial state
  x[:, 0] = x0.flatten()

  # Solve LQR using simple tools --> Taken from HW6 quadrotor.py file
  u_normal = lqr_normal(A,B,Q,R,x)
  print(u_normal)

  # Solve LQR as an optimization problem
  K, _ = LinearQuadraticRegulator(A,B,Q,R)
  u_drake = lqr_drake(K,x,T_steps)
  print(u_drake)